In [42]:
!pip install torchtext

^C


In [44]:
import bz2
import os.path
import numpy as np
import re

import torchtext
from torchtext.data import get_tokenizer
from torchtext.legacy.data import Field
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torchtext.vocab import FastText, vocab
import torch

import sys
from tqdm import tqdm
from datasets import load_dataset, list_metrics
from datasets import IterableDataset
#from datasets.IterableDataset import shuffle
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

from torch.optim import Adam
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM


In [ ]:
from datasets import load_dataset
dataset = load_dataset('amazon_polarity')#, split='train')
#shuffled_dataset = dataset.shuffle(seed=42)
#small_dataset = shuffled_dataset.select(range(36000))
#len(small_dataset)
#print(dataset)
#small_dataset



Reusing dataset amazon_polarity (C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4)
100%|██████████| 2/2 [00:01<00:00,  1.70it/s]


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
def tokenize_function(examples):
    return tokenizer(examples["content"], padding="max_length", truncation=True)


#tokenized_datasets

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\Jonah/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve

In [ ]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(50)) 
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(50)) 
big_train_dataset = dataset["train"].shuffle(seed=42).select(range(3000)) 
big_eval_dataset = dataset["test"].shuffle(seed=42).select(range(3000)) 

Loading cached shuffled indices for dataset at C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4\cache-ea9e58304cf21e85.arrow
Loading cached shuffled indices for dataset at C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4\cache-951db2bd067ed241.arrow
Loading cached shuffled indices for dataset at C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4\cache-ea9e58304cf21e85.arrow
Loading cached shuffled indices for dataset at C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4\cache-951db2bd067ed241.arrow


In [ ]:
tokenized_small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
tokenized_small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)
tokenized_big_train_dataset = big_train_dataset.map(tokenize_function, batched=True)
tokenized_big_eval_dataset = big_eval_dataset.map(tokenize_function, batched=True)


Loading cached processed dataset at C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4\cache-11bfeacb8a48b394.arrow
Loading cached processed dataset at C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4\cache-6370f14e6d24b1ae.arrow
100%|██████████| 3/3 [00:01<00:00,  2.12ba/s]
Loading cached processed dataset at C:\Users\Jonah\.cache\huggingface\datasets\amazon_polarity\amazon_polarity\3.0.0\56923eeb72030cb6c4ea30c8a4e1162c26b25973475ac1f44340f0ec0f2936f4\cache-c630d837a3ff7df2.arrow


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\Jonah/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased

In [ ]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments("test_trainer")

trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_small_train_dataset, eval_dataset=tokenized_small_eval_dataset
)

In [ ]:
trainer.train()


The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: content, title.
***** Running training *****
  Num examples = 50
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 21
100%|██████████| 21/21 [15:40<00:00, 36.40s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 21/21 [15:40<00:00, 44.80s/it]

{'train_runtime': 940.8238, 'train_samples_per_second': 0.159, 'train_steps_per_second': 0.022, 'train_loss': 0.5976756867908296, 'epoch': 3.0}


TrainOutput(global_step=21, training_loss=0.5976756867908296, metrics={'train_runtime': 940.8238, 'train_samples_per_second': 0.159, 'train_steps_per_second': 0.022, 'train_loss': 0.5976756867908296, 'epoch': 3.0})

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("./checkpoint-6000")
training_args = TrainingArguments("test_trainer")

trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_small_train_dataset, eval_dataset=tokenized_small_eval_dataset
)
predictions = trainer.predict(tokenized_small_eval_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)

loading configuration file ./checkpoint-6000\config.json
Model config BertConfig {
  "_name_or_path": "./checkpoint-6000",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file ./checkpoint-6000\pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassificat

(50, 2) (50,)


In [49]:
preds = np.argmax(predictions.predictions, axis=-1)
metric = load_metric("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.94, 'f1': 0.9454545454545454}

In [46]:
list_metrics()

['accuracy',
 'bertscore',
 'bleu',
 'bleurt',
 'cer',
 'chrf',
 'code_eval',
 'comet',
 'competition_math',
 'coval',
 'cuad',
 'f1',
 'gleu',
 'glue',
 'google_bleu',
 'indic_glue',
 'matthews_correlation',
 'meteor',
 'pearsonr',
 'precision',
 'recall',
 'rouge',
 'sacrebleu',
 'sari',
 'seqeval',
 'spearmanr',
 'squad',
 'squad_v2',
 'super_glue',
 'ter',
 'wer',
 'wiki_split',
 'xnli']

In [41]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    #tokenizer=tokenizer,
)
trainer.evaluate()

100%|██████████| 7/7 [5:40:16<00:00, 2916.70s/it]
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: content, title.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 8


ValueError: You have to specify either input_ids or inputs_embeds